# Pocc quality assessment feature

## 1. Import code libraries

In [ ]:
# Standard code libraries
import os
import platform
import glob

import scipy
import numpy as np

import matplotlib.pyplot as plt
import ipywidgets as widgets

%matplotlib widget

In [ ]:
# Custom code libraries from the ReSurfEMG repository
import resurfemg.preprocessing.ecg_removal as ecg_rm
import resurfemg.preprocessing.envelope as evl
import resurfemg.preprocessing.filtering as filt

from resurfemg.data_connector.tmsisdk_lite import Poly5Reader

from resurfemg.pipelines.pipelines import ecg_removal_gating

from resurfemg.config.config import Config

## Initiation of output folder for experiments

In [ ]:
# Root directory for patient data
config = Config()
root_patient_data_directory = \
    config.get_directory('test_data')

if platform.system() == 'Windows':
    path_sep = "\\"
else:
    path_sep = '/'

patient_idx = 0

## 2. Load the ventilator and sEMG data

In [ ]:
# Identify all recordings available for the selected patient/measurement_date

emg_pattern = os.path.join(root_patient_data_directory, '**/*.Poly5')
emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
vent_files = []

for file in emg_and_vent_files:
    if 'vent' in file:
        vent_files.append(file)
    else:
        emg_files.append(file)

emg_file_chosen = emg_files[0]
vent_file_chosen = vent_files[0]
print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
vent_fs = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples
y_vent = data_vent_samples

# Define the time axes
t_emg = [i/fs_emg for i in range(len(y_emg[0, :]))]
t_vent = [i/vent_fs for i in range(len(y_vent[0, :]))]

# Default settings for window of interest including the end-expiratory occlusion
# manoeuvres (Pocc)
t_start_default = t_vent[-1]-60
t_end_default = t_vent[-1]-5

## 3. Select the time window of interest

Enter the start and end time of the window in which the occlusions occur. Make sure to take some margin around the peaks in the order of a couple of seconds to allow for the filtering algorithms to work properly. On the other hand, if the window is to broad, the algorithms will also take longer to evaluate.

In [ ]:
# Window selection

start_widget = widgets.BoundedFloatText(
    value=np.floor(t_start_default),
    min=1,
    max=t_vent[-1],
    step=1,
    description='Start:',
    disabled=False
)
end_widget = widgets.BoundedFloatText(
    value=np.floor(t_end_default),
    min=1,
    max=t_vent[-1],
    step=1,
    description='End:',
    disabled=False
)
widgets.VBox([start_widget, end_widget])

In [ ]:
# Plot the selected window if valid start and end times are entered
start = start_widget.value  
end = end_widget.value  
if ((0 > start) | (t_vent[-1] < start) | (0 > end) | (t_vent[-1] < end) 
    | (start >= end)):    
    print('Please make sure that 1) the start and end times are within the'  
          +' limits of the recording and 2) that the start time is before the' 
          + ' end time.')
else:
    # Plot the selected window.
    start_idx = int(float(start)* fs_emg)
    end_s = min([int(float(end)*fs_emg), len(y_emg[0,:])-1])
    start_vent_s = int(float(start)* vent_fs)
    end_vent_s = min(
        [int(float(end)* vent_fs), len(y_vent[0,:])-1]
    )

    fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(12, 6))
    axis[0, 0].set(title='sEMG leads')
    axis[0, 0].grid(True)
    axis[0, 0].plot(t_emg[int(start_idx):int(end_s)], 
                    y_emg[0][int(start_idx):int(end_s)])
    axis[0, 0].set_ylabel('ECG (uV)')
    axis[1, 0].plot(t_emg[int(start_idx):int(end_s)], 
                    y_emg[1][int(start_idx):int(end_s)])
    axis[1, 0].set_ylabel('sEMGdi (uV)')
    axis[1, 0].set_xlabel('t (s)')
    
    axis[2, 0].axis('off')

    axis[0, 1].set(title='Ventilator data')
    axis[0, 1].grid(True)
    axis[0, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[0][int(start_vent_s):int(end_vent_s)]
    )
    axis[0, 1].set_ylabel('Paw (cmH2O)')
    axis[1, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[1][int(start_vent_s):int(end_vent_s)]
    )
    axis[1, 1].set_ylabel('F (L/min)')
    axis[2, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[2][int(start_vent_s):int(end_vent_s)]
    )
    axis[2, 1].set_ylabel('V (mL)')
    axis[2, 1].set_xlabel('t (s)')

Examine at the selected window:
1. Does it contain all occlusions? If not, adapt the start and/or end time to include all Pocc manoeuvres.
2. Does the window contain limited data before and after the occlusions (~10 s). If noy, you can adapt the start and/or end time to optimise calculation speed.

## 4. Pre-process the sEMGdi

In [ ]:
# Set the default pipeline parameters
# Gating settings
gate_width_default = 0.10
gate_threshold_default = 0.30
gate_ECG_shift_default = -10
gate_twice = False

# RMS window
RMS_window_ms_default = 200

# # Peak detection settings
# time_shift_default = 0.5 - RMS_window_ms_default/1000/2
P_occ_prominence_factor_default = 0.8
# sEAdi_prominence_factor_default = 0.5

In [ ]:
# Eliminate the baseline wander from the data using a band-pass filter
bd_filtered_file_data = filt.emg_bandpass_butter_sample(
    data_emg_samples, 1, 500, 2048, output='sos')
    
processed_data_emg_di = bd_filtered_file_data[1]

In [ ]:
# Gating settings

# Window size to plot the gating results from
plot_window = 5         

# QRS peak detection settings
# peak_fraction = 0.30 (default value)
peak_fraction = gate_threshold_default     # Minimal relative peak height
peak_width = 0.001
peak_dist = int(fs_emg/3)

# Shift of gated in samples relative to detected peaks      
# ECG_shift = -10 (default value)
ECG_shift = gate_ECG_shift_default

# Gate width in seconds
# gate_width = 0.10 (default value)
gate_width = gate_width_default

Apply QRS gating

In [ ]:
# First run of QRS gating

# Detect ECG peaks on 1Hz high-pass filtered signals
ecg = bd_filtered_file_data[0]
ecg_rms = evl.full_rolling_rms(ecg, 10)
max_ecg_rms = max(
    ecg_rms[int(start_idx):int(start_idx+plot_window*fs_emg)])
min_ecg_rms = min(
    ecg_rms[int(start_idx):int(start_idx+plot_window*fs_emg)])
peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

ECG_peaks, properties  = scipy.signal.find_peaks(
    ecg_rms, 
    height=peak_height, 
    width=peak_width*fs_emg, 
    distance=peak_dist
)

# The RMS filter is not centred. Therefore, shift the ECG peak locations
ECG_peaks = ECG_peaks - ECG_shift

# Filter EXG signals 20Hz high pass
ECG_h20 = filt.emg_bandpass_butter_sample(ecg, 20, 500, 2048, output='sos')
emg_di_h20 = filt.emg_bandpass_butter_sample(
    processed_data_emg_di, 20, 500, 2048, output='sos')

# Gate ECG and EMG signal
# Fill methods 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
# 3: Moving average
gate_fill_method = 3    
gate_width_samples = int(gate_width*fs_emg)

gate_samples = list()
for i in range(len(ECG_peaks)):
    for k in range(
        int(ECG_peaks[i]-gate_width_samples/2),
        int(ECG_peaks[i]+gate_width_samples/2)
    ):
        gate_samples.append(k)

ECG_gated = ecg_rm.gating(
    ECG_h20, ECG_peaks, gate_width=gate_width_samples, method=3)
emg_di_gated = ecg_rm.gating(
    emg_di_h20, ECG_peaks, gate_width=gate_width_samples, method=3)


In [ ]:
ecg_peaks_fun = ecg_rm.detect_ecg_peaks(
    ecg_raw=data_emg_samples[0],
    fs=fs_emg,
    bp_filter=True,
)

ecg_gated_fun = ecg_removal_gating(
    ECG_h20,
    ecg_peaks_fun,
    gate_width_samples, 
    ecg_shift=10,
    )

emg_di_gated_fun = ecg_removal_gating(
    emg_di_h20,
    ecg_peaks_fun,
    gate_width_samples,
    ecg_shift=10,
    )

np.sum(np.abs(emg_di_gated-emg_di_gated_fun))

In [ ]:
# Plot gating result
fig, axis = plt.subplots(nrows=4, ncols=1, figsize=(12, 6))
start_i = int(start_idx)
end_i = int(start_idx+plot_window*2048)
axis[0].grid(True)
axis[0].plot(t_emg[start_i:end_i], bd_filtered_file_data[0, start_i:end_i])
axis[0].set_ylabel('raw ECG (uV)')
axis[0].set(title='leads in EMG')
axis[1].grid(True)
axis[1].plot(t_emg[start_i:end_i], ECG_gated[start_i:end_i])
axis[1].set_ylabel('ECG gated (uV)')
axis[2].grid(True)
axis[2].plot(t_emg[start_i:end_i], emg_di_gated[start_i:end_i], color='tab:blue')
axis[2].plot(t_emg[start_i:end_i], emg_di_gated_fun[start_i:end_i], color='tab:orange')
axis[2].set_ylabel('EMGdi gated (uV)')
axis[3].grid(True)
axis[3].plot(t_emg[start_i:end_i], ecg_rms[start_i:end_i])
axis[3].set_ylabel('ECG rms (uV)')
axis[3].hlines(
    y=peak_height, 
    xmin=t_emg[start_i], 
    xmax=t_emg[end_i],
    color = "C1"
)
axis[3].set_xlabel('t (s)')

for idx in range(len(ECG_peaks)):
    if ((ECG_peaks[idx] > int(start_idx)) 
        & (ECG_peaks[idx] < int(start_idx+plot_window*2048))):
        axis[3].plot(t_emg[ECG_peaks[idx]], ecg_rms[ECG_peaks[idx]],'rx')


In [ ]:
# Second run of QRS gating, if selected with gate_twice == True
# This is usefull, e.g., in case of splitted QRS complex or pacing

# Detect ECG peaks on 1Hz high-pass filtered signals
ecg = ecg_gated_fun
ecg_rms = evl.full_rolling_rms(ecg_gated_fun, 10)
max_ecg_rms = max(ecg_rms)
min_ecg_rms = min(ecg_rms)
peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

ECG_peaks_2nd, properties  = scipy.signal.find_peaks(
    ecg_rms, 
    height=peak_height, 
    width=peak_width*fs_emg, 
    distance=peak_dist
)
# The RMS filter is not centred. Therefore, shift the ECG peak locations
ECG_peaks_2nd = ECG_peaks_2nd - ECG_shift

# Gate ECG and EMG signal
# Fill methods: 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
# 3: Moving average
gate_fill_method = 3    
gate_width_samples = int(gate_width*fs_emg)

gate_samples = list()
for i in range(len(ECG_peaks)):
    for k in range(
        int(ECG_peaks[i]-gate_width_samples/2),
        int(ECG_peaks[i]+gate_width_samples/2)
    ):
        gate_samples.append(k)


ECG_gated_twice = ecg_rm.gating(
    ecg_gated_fun, ECG_peaks_2nd, gate_width=gate_width_samples, method=3)
emg_di_gated_twice = ecg_rm.gating(
    emg_di_gated_fun, ECG_peaks_2nd, gate_width=gate_width_samples, method=3)

In [ ]:
ecg_peaks_fun_twice = ecg_rm.detect_ecg_peaks(
    ecg_raw=ecg_gated_fun,
    fs=fs_emg,
    bp_filter=False,
)

ecg_gated_fun_twice = ecg_removal_gating(
    ecg_gated_fun,
    ecg_peaks_fun_twice,
    gate_width_samples, 
    ecg_shift=10,
    )

emg_di_gated_fun_twice = ecg_removal_gating(
    emg_di_gated_fun,
    ecg_peaks_fun_twice,
    gate_width_samples,
    ecg_shift=10,
    )

np.sum(np.abs(emg_di_gated_twice-emg_di_gated_fun_twice))

In [ ]:
# Plot gating result
fig, axis = plt.subplots(nrows=4, ncols=1, figsize=(12, 6))
start_i = int(start_idx)
end_i = int(start_idx+plot_window*2048)
axis[0].grid(True)
axis[0].plot(t_emg[start_i:end_i], bd_filtered_file_data[0, start_i:end_i])
axis[0].set_ylabel('raw ECG (uV)')
axis[0].set(title='leads in EMG')
axis[1].grid(True)
axis[1].plot(t_emg[start_i:end_i], ECG_gated_twice[start_i:end_i], color='tab:blue')
axis[1].plot(t_emg[start_i:end_i], ecg_gated_fun_twice[start_i:end_i], color='tab:orange')
axis[1].set_ylabel('ECG gated (uV)')
axis[2].grid(True)
axis[2].plot(t_emg[start_i:end_i], emg_di_gated_twice[start_i:end_i], color='tab:blue')
axis[2].plot(t_emg[start_i:end_i], emg_di_gated_fun_twice[start_i:end_i], color='tab:orange')
axis[2].set_ylabel('EMGdi gated (uV)')
axis[3].grid(True)
axis[3].plot(t_emg[start_i:end_i], ecg_rms[start_i:end_i])
axis[3].set_ylabel('ECG rms (uV)')
axis[3].hlines(
    y=peak_height, 
    xmin=t_emg[start_i], 
    xmax=t_emg[end_i],
    color = "C1"
)
axis[3].set_xlabel('t (s)')

for idx in range(len(ECG_peaks)):
    if ((ECG_peaks[idx] > int(start_idx)) 
        & (ECG_peaks[idx] < int(start_idx+plot_window*2048))):
        axis[3].plot(t_emg[ECG_peaks[idx]], ecg_rms[ECG_peaks[idx]],'rx')
